In [1]:
# TensorFlow での線形回帰を行う

# 入出力とも多次元配列であるテンソルと呼ばれる形で出力
# tensorFlowが優れているのはGPUが使える点
# 以下お決まり設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


In [3]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
# housing.data : データの中身を見れる(多次元配列)
# hoousing.data.shape : 配列の定義を見れる ここでは(20640,8) 20640インスタンス、特徴量8個

housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
# np.ones((m, 1)) : [1.]の20640個の配列
# np.c_ : 行列の連結
# np.c_[np.ones((m, 1)), housing.data] : [1., housing.dataの特徴量(8個)] の20640インスタンスのnumpy配列が出来上がる
# 先頭にバイアスとして　１　が設定されるnumpy配列ができているということ

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
# 上記の結果、このnumpy配列は(20640,9)となっている。ちなみにtf.constant なので定数

y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
# housing.target : (20640,) のnumpy配列
# 配列のshapeを指定する際に (n, -1) のように-1を指定すると要素数に合わせてn × mの2次元配列となる
# このmはもとの配列の要素数に応じて変化する
# housing.target.reshape(-1, 1) の場合は　列数を１に指定、行数は -1 のため、列数1に合わせてreshapeする　⇒ (20640,1)

XT = tf.transpose(X) # transpose() は転置行列への変換 ⇒ (9,20640) になる
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
# tf.matmul : 行列積 
# tf.matrix_inverse : 逆行列

with tf.Session() as sess:
    theta_value = theta.eval()

In [22]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [23]:
# ちなみにscikit-learnで組むと以下のようになる
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]
